In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import io
import base64
import time
import warnings
warnings.filterwarnings('ignore')

# Install required packages first
try:
    import easyocr
    from IPython.display import display, clear_output
    import ipywidgets as widgets
except ImportError:
    print("Installing required packages...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python", "pillow", "easyocr", "matplotlib", "ipywidgets", "seaborn", "pandas", "numpy"])
    print("Packages installed successfully! Please restart the kernel and run again.")
    exit()

class ColabImageTextTranslator:
    def __init__(self):
        self.setup_translation_dataset()
        self.setup_ml_model()
        self.setup_gui()
        print("Translation from Image initialized successfully!")

    def setup_translation_dataset(self):
        """Load English to French translation dataset from text files"""
        try:
            # Read English words from file
            with open('_english_.txt', 'r', encoding='utf-8') as f:
                english_words = [line.strip().lower() for line in f.readlines() if line.strip()]

            # Read French translations from file
            with open('_french_.txt', 'r', encoding='utf-8') as f:
                french_words = [line.strip() for line in f.readlines() if line.strip()]

            # Create translation dictionary
            self.translation_dict = {}
            for eng, fr in zip(english_words, french_words):
                self.translation_dict[eng] = fr

            print(f"Translation dictionary loaded with {len(self.translation_dict)} words from text files")

            # Display sample of loaded words
            print("Words are loaded:")
            sample_items = list(self.translation_dict.items())[:5]
            for eng, fr in sample_items:
                print(f"   {eng} → {fr}")

        except FileNotFoundError as e:
            print(f"Error: Could not find translation files: {e}")
            print("Please make sure '_english_.txt' and '_french_.txt' are in the current directory")
            # Fallback to a small default dictionary
            self.translation_dict = {"hello": "bonjour", "world": "monde"}
        except Exception as e:
            print(f"Error loading translation dataset: {e}")
            self.translation_dict = {"hello": "bonjour", "world": "monde"}

    def setup_ml_model(self):
        """Setup machine learning model for text recognition"""
        try:
            # Initialize EasyOCR reader with error handling
            print("Initializing EasyOCR reader...")
            self.reader = easyocr.Reader(['en'])
            self.model_loaded = True
            print("ML Text Recognition model ready (EasyOCR)")
        except Exception as e:
            print(f"Error setting up ML model: {e}")
            self.model_loaded = False
            self.reader = None

    def setup_gui(self):
        """Setup the GUI using ipywidgets"""
        # Create widgets
        self.title = widgets.HTML(
            "<h1 style='color: #2E86AB; text-align: center;'>Translation from Image - English to French</h1>"
        )

        # Upload section
        self.upload_btn = widgets.FileUpload(
            description='Upload Image',
            multiple=False,
            accept='image/*'
        )

        # Process buttons
        self.extract_btn = widgets.Button(description='Extract Text')
        self.translate_btn = widgets.Button(description='Translate')
        self.clear_btn = widgets.Button(description='Clear All')

        # Analysis buttons
        self.stats_btn = widgets.Button(description='Show Analysis')
        self.dict_btn = widgets.Button(description='View Dictionary')

        # Output areas
        self.image_output = widgets.Output()
        self.extracted_output = widgets.Output()
        self.translated_output = widgets.Output()
        self.status_output = widgets.Output()

        # Connect events
        self.upload_btn.observe(self.on_upload, names='value')
        self.extract_btn.on_click(self.on_extract)
        self.translate_btn.on_click(self.on_translate)
        self.clear_btn.on_click(self.on_clear)
        self.stats_btn.on_click(self.on_stats)
        self.dict_btn.on_click(self.on_dictionary)

        # Layout
        button_box1 = widgets.HBox([self.extract_btn, self.translate_btn, self.clear_btn])
        button_box2 = widgets.HBox([self.stats_btn, self.dict_btn])

        # Display the GUI
        display(self.title)
        display(widgets.HTML("<h3>Upload Image</h3>"))
        display(self.upload_btn)
        display(widgets.HTML("<h3>Image Preview</h3>"))
        display(self.image_output)
        display(widgets.HTML("<h3>Processing</h3>"))
        display(button_box1)
        display(widgets.HTML("<h3>Results</h3>"))
        display(widgets.HTML("<h4>Extracted English Text:</h4>"))
        display(self.extracted_output)
        display(widgets.HTML("<h4>Translated French Text:</h4>"))
        display(self.translated_output)
        display(widgets.HTML("<h3>Additional Features</h3>"))
        display(button_box2)
        display(widgets.HTML("<h4>Status:</h4>"))
        display(self.status_output)

        # Initialize state
        self.current_image = None
        self.extracted_english_text = ""

        with self.status_output:
            print("Ready! Upload an image to begin...")
            print(f"Loaded {len(self.translation_dict)} words from your dictionary files")
            if hasattr(self, 'reader') and self.reader:
                print("Text extraction model: READY")
            else:
                print("Text extraction model: NOT AVAILABLE")

    def on_upload(self, change):
        """Handle image upload"""
        if not change['new']:
            return

        try:
            uploaded = list(change['new'].values())[0]
            image_data = uploaded['content']
            self.current_image = Image.open(io.BytesIO(image_data))

            # Display image
            self.image_output.clear_output()
            with self.image_output:
                plt.figure(figsize=(10, 6))
                plt.imshow(self.current_image)
                plt.axis('off')
                plt.title('Uploaded Image')
                plt.show()

            self.update_status(f"Image uploaded successfully: {uploaded['name']}")

        except Exception as e:
            self.update_status(f"Error uploading image: {e}")

    def on_extract(self, btn):
        """Extract text from image"""
        if not self.current_image:
            self.update_status("Please upload an image first")
            return

        if not hasattr(self, 'reader') or self.reader is None:
            self.update_status("Text extraction model not available.")
            return

        try:
            self.update_status("Extracting text from image...")
            opencv_image = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2BGR)
            results = self.reader.readtext(opencv_image)

            extracted_texts = []
            for (bbox, text, confidence) in results:
                if confidence > 0.3 and self.is_english_text(text):
                    extracted_texts.append(text)

            self.extracted_english_text = " ".join(extracted_texts)

            self.extracted_output.clear_output()
            with self.extracted_output:
                if self.extracted_english_text:
                    print("Extracted Text:")
                    print("-" * 40)
                    print(self.extracted_english_text)
                    print(f"\nFound {len(extracted_texts)} text blocks")

                    words_in_text = self.extracted_english_text.lower().split()
                    words_in_dict = [word for word in words_in_text
                                   if ''.join(char for char in word if char.isalnum()).lower() in self.translation_dict]
                    print(f"{len(words_in_dict)} words from your dictionary detected")
                else:
                    print("No English text detected in the image.")
                    print("Try images with clear English text")

            if self.extracted_english_text:
                self.update_status(f"Text extracted successfully ({len(extracted_texts)} text blocks)")
            else:
                self.update_status("No English text detected")

        except Exception as e:
            self.update_status(f"Error extracting text: {e}")

    def on_translate(self, btn):
        """Translate extracted text to French"""
        if not self.extracted_english_text:
            self.update_status("Please extract text first")
            return

        try:
            self.update_status("Translating text to French...")
            words = self.extracted_english_text.split()
            translated_words = []
            translation_details = []

            for word in words:
                clean_word = ''.join(char for char in word if char.isalnum()).lower()
                if clean_word in self.translation_dict:
                    translated_words.append(self.translation_dict[clean_word])
                    translation_details.append(f"{word} → {self.translation_dict[clean_word]}")
                else:
                    translated_words.append(f"[{word}]")
                    translation_details.append(f"{word} → [Not in dictionary]")

            translated_text = " ".join(translated_words)

            self.translated_output.clear_output()
            with self.translated_output:
                print("Translated Text:")
                print("-" * 40)
                print(translated_text)

                total_words = len(words)
                translated_count = len([w for w in translated_words if not w.startswith('[')])
                coverage = (translated_count / total_words) * 100 if total_words > 0 else 0

                print(f"\nTranslation Statistics:")
                print(f"Coverage: {coverage:.1f}% ({translated_count}/{total_words} words)")
                print(f"Dictionary size: {len(self.translation_dict)} words")
                print(f"Untranslated words marked with [brackets]")

                if translation_details:
                    print(f"\nTranslation Details (first 5 words):")
                    for detail in translation_details[:5]:
                        print(f"  {detail}")

            self.update_status("Translation completed successfully")

        except Exception as e:
            self.update_status(f"Error during translation: {e}")

    def on_clear(self, btn):
        """Clear all outputs"""
        self.current_image = None
        self.extracted_english_text = ""
        self.image_output.clear_output()
        self.extracted_output.clear_output()
        self.translated_output.clear_output()
        self.status_output.clear_output()

        with self.status_output:
            print("All cleared! Upload a new image to begin...")
            print(f"Dictionary ready with {len(self.translation_dict)} words")
            if hasattr(self, 'reader') and self.reader:
                print("Text extraction model: READY")

    def on_stats(self, btn):
        """Show text analysis"""
        if not self.extracted_english_text:
            self.update_status("No text available for analysis")
            return

        self.status_output.clear_output()
        with self.status_output:
            words = self.extracted_english_text.split()
            word_count = len(words)
            char_count = len(self.extracted_english_text)
            unique_words = len(set(words))

            print("TEXT ANALYSIS")
            print("=" * 50)
            print(f"Total words: {word_count}")
            print(f"Total characters: {char_count}")
            print(f"Unique words: {unique_words}")
            print(f"Average word length: {char_count/word_count:.1f} characters")

            word_freq = {}
            for word in words:
                clean_word = word.lower().strip('.,!?;:')
                if clean_word:
                    word_freq[clean_word] = word_freq.get(clean_word, 0) + 1

            sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
            print(f"\nTop 5 Frequent Words:")
            for word, freq in sorted_words:
                in_dict = "✓" if word in self.translation_dict else "✗"
                print(f"  {in_dict} '{word}': {freq} times")

            words_in_dict = sum(1 for word in words
                              if ''.join(char for char in word if char.isalnum()).lower() in self.translation_dict)
            coverage = (words_in_dict / word_count) * 100 if word_count > 0 else 0
            print(f"\nDictionary Coverage for this text: {coverage:.1f}%")

    def on_dictionary(self, btn):
        """Show translation dictionary"""
        self.status_output.clear_output()
        with self.status_output:
            print("TRANSLATION DICTIONARY")
            print("=" * 50)
            print(f"Total words: {len(self.translation_dict)}")
            print(f"Source: _english_.txt & _french_.txt files")

            sorted_dict = sorted(self.translation_dict.items())
            print("\nDictionary Entries (A-Z):")
            print("-" * 40)

            for i in range(0, min(20, len(sorted_dict)), 2):
                line = ""
                if i < len(sorted_dict):
                    eng1, fr1 = sorted_dict[i]
                    line += f"• {eng1:12} → {fr1:15}"
                if i + 1 < len(sorted_dict):
                    eng2, fr2 = sorted_dict[i + 1]
                    line += f"   • {eng2:12} → {fr2:15}"
                print(line)

            if len(self.translation_dict) > 20:
                print(f"\n... and {len(self.translation_dict) - 20} more words")

    def is_english_text(self, text):
        """Basic check if text appears to be English"""
        clean_text = ''.join(char for char in text if char.isalnum() or char.isspace()).lower()

        if not clean_text.strip():
            return False

        common_english_words = ['the', 'and', 'is', 'in', 'to', 'of', 'a', 'for', 'on', 'with', 'by']
        words = clean_text.split()

        if len(words) == 0:
            return False

        for word in words:
            if word in common_english_words:
                return True

        dict_words_found = sum(1 for word in words
                             if ''.join(char for char in word if char.isalnum()).lower() in self.translation_dict)
        if dict_words_found > 0:
            return True

        alpha_count = sum(1 for char in clean_text if char.isalpha())
        total_chars = len(clean_text)

        if total_chars > 0 and alpha_count / total_chars > 0.7:
            return True

        return len(clean_text) > 2

    def update_status(self, message):
        """Update status output"""
        self.status_output.clear_output()
        with self.status_output:
            print(f"{message}")

# Main execution
print("Starting Translation from image with Your Dictionary Files...")

# Create and display the application
translator_app = ColabImageTextTranslator()

Starting Translation from image with Your Dictionary Files...
Translation dictionary loaded with 62 words from text files
Words are loaded:
   hello → bonjour
   world → monde
   good → bon
   morning → matin
   evening → soir
Initializing EasyOCR reader...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteML Text Recognition model ready (EasyOCR)


HTML(value="<h1 style='color: #2E86AB; text-align: center;'>Translation from Image - English to French</h1>")

HTML(value='<h3>Upload Image</h3>')

FileUpload(value={}, accept='image/*', description='Upload Image')

...

Output()

HTML(value='<h3>Additional Features</h3>')

HTML(value='<h4>Status:</h4>')

Output()

Translation from Image initialized successfully!
